In [1]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_foursquare=pd.read_csv('data/api_foursquare.csv')
df_foursquare=df_foursquare.drop(columns=['Unnamed: 0'])
df_foursquare['closed_bucket'].unique()

array(['LikelyOpen', 'VeryLikelyOpen', 'Unsure'], dtype=object)

In [3]:
lista_lugares=[]
for fila in df_foursquare.values:
    lista_lugares.append(tuple(fila))

In [4]:
df_aemet=pd.read_csv('data/df_aemet_final.csv')
df_aemet.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


In [5]:
df_aemet['localizacion_id'] = df_aemet['localizacion_id'].str.replace(r'-id\d+', '', regex=True)
df_aemet.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la


In [6]:
df_aemet['fecha']=pd.to_datetime(df_aemet['fecha'])

In [7]:
df_municipios=pd.read_csv('data/municipios.csv')
df_municipios.head(1)

,Unnamed: 0,acebeda-la,ajalvir,alameda-del-valle,alamo-el,alcala-de-henares,alcobendas,alcorcon,aldea-del-fresno,algete,...,villamanta,villamantilla,villanueva-de-la-canada,villanueva-del-pardillo,villanueva-de-perales,villar-del-olmo,villarejo-de-salvanes,villaviciosa-de-odon,villavieja-del-lozoya,zarzalejo
0,0,41.0869583,40.5342302,40.9177178,40.2307296,40.4818396,40.5400082,40.3492998,40.3215212,40.5961652,...,40.2990939,40.3421033,40.4472106,40.4949232,40.3466066,40.3364465,40.1683582,40.3573787,41.0063622,40.5483016


In [8]:
df_municipios=df_municipios.T
df_municipios=df_municipios.reset_index()
df_municipios=df_municipios[1:]
df_municipios.head(1)

,index,0,1,2
1,acebeda-la,41.0869583,-3.624399,La Acebeda


In [9]:
df_aemet = df_aemet[df_aemet['localizacion_id'].isin(set(df_municipios['index'].unique()))]

In [10]:
df_municipios.rename(columns={0: 'latitud'}, inplace=True)
df_municipios.rename(columns={1: 'longitud'}, inplace=True)

In [11]:
lista_datosmeteo=[]
for fila in df_aemet.values:
    lista_datosmeteo.append(tuple(fila))


In [12]:
df_municipios['latitud']=df_municipios['latitud'].astype(float)
df_municipios['longitud']=df_municipios['longitud'].astype(float)
df_municipios.head(1)

,index,latitud,longitud,2
1,acebeda-la,41.086958,-3.624399,La Acebeda


In [13]:
lista_municipios=[]

for fila in df_municipios.values:
    lista_municipios.append(tuple(fila))


In [14]:
try:
    conn = psycopg2.connect(
    database="laaboratorio3",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")

In [15]:
conn.rollback()

conn = conn.cursor()
conn

<cursor object at 0x0000022588157E60; closed: 0>

In [17]:
# Crear tablas
# Tabla Municipio
# index        object
# latitud     float64
# longitud    float64
# 2            object

querydecreacion1 = """
    CREATE TABLE IF NOT EXISTS Municipios (
    id_municipio VARCHAR(50) PRIMARY KEY,
    latitud DECIMAL,
    longitud DECIMAL,
    nombre VARCHAR(50)
);
"""

# Ejecutar la consulta y confirmar los cambios
conn.execute(querydecreacion1)
conn.commit()

# Tabla Lugar

# municipio         object
# category          object
# fsq_id            object
# closed_bucket     object
# distance         float64
# link              object
# name              object
# address           object
# latitude         float64
# longitude        float64
# dtype: object

querydecreacion2 = """
    CREATE TABLE IF NOT EXISTS Lugar (
    id_lugar SERIAL PRIMARY KEY,
    id_municipio VARCHAR,
    categoria VARCHAR,
    fsq_id VARCHAR,
    closed_bucket VARCHAR,
    distance DECIMAL,
    link VARCHAR,
    name VARCHAR,
    address VARCHAR,
    latitud DECIMAL,
    longitud DECIMAL,
    FOREIGN KEY (id_municipio) REFERENCES Municipios(id_municipio)
    on delete restrict
    on update cascade
);
"""

# Ejecutar la consulta y confirmar los cambios
conn.execute(querydecreacion2)
conn.commit()

# Tabla Tiempo

# fecha                     datetime64
# cielo                     object
# temp._(°c)                 int64
# sen._térmica_(°c)          int64
# racha_máx._(km/h)        float64
# precipitación_(mm)       float64
# nieve_(mm)                 int64
# humedad_relativa_(%)       int64
# prob._precip.__(%)        object
# prob._de_nieve_(%)         int64
# prob._de_tormenta_(%)      int64
# avisos                    object
# dirección_viento          object
# velocidad_del_viento       int64
# localizacion_id           object

querydecreacion3 = """
    CREATE TABLE IF NOT EXISTS Tiempo (
    id_tiempo SERIAL PRIMARY KEY,
    fecha DATE,
    cielo VARCHAR,
    temperatura INT,
    sensacion_termica INT,
    racha_max DECIMAL,
    precipitacion DECIMAL,
    nieve INT,
    humedad_relativa INT,
    prob_precip VARCHAR,
    prob_de_nieve INT,
    prob_de_tormenta INT,
    avisos VARCHAR,
    direccion_viento VARCHAR,
    velocidad_viento INT,
    id_municipio VARCHAR,
    FOREIGN KEY (id_municipio) REFERENCES Municipios(id_municipio)
    on delete restrict
    on update cascade
);
"""

# Ejecutar la consulta y confirmar los cambios
conn.execute(querydecreacion3)
conn.commit()

TypeError: function missing required argument 'vars_list' (pos 2)

In [ ]:
conn.rollback()

conn = conn.cursor()
conn

<cursor object at 0x000001EFD4532500; closed: 0>

In [ ]:
queryinsercion = 'INSERT INTO municipios (id_municipio, latitud, longitud, nombre) VALUES (%s,%s,%s,%s);'
listavaloresinsercion=lista_municipios
conn.executemany(queryinsercion,listavaloresinsercion)
conn.commit()

In [ ]:
conn.rollback()

conn = conn.cursor()
conn

In [ ]:
queryinsercion = "INSERT INTO lugar (id_municipio, categoria, fsq_id, closed_bucket, distance, link, name, address, latitud, longitud) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_lugares
conn.executemany(queryinsercion,listavaloresinsercion)
conn.commit()

In [ ]:
conn.rollback()

conn = conn.cursor()
conn

In [ ]:
queryinsercion = "INSERT INTO tiempo (fecha, cielo, temperatura, sensacion_termica, racha_max, precipitacion, nieve, humedad_relativa, prob_precip, prob_de_nieve, prob_de_tormenta, avisos, direccion_viento, velocidad_viento, id_municipio) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
listavaloresinsercion = lista_datosmeteo
conn.executemany(queryinsercion, listavaloresinsercion)
conn.commit()